Now we have enough data

In [1]:
import main, pickle

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
random_board_pairs = []
with open("2021-09-09-mcts-v3-boards10-random.pickle", "rb") as f:
    try:
        while True:
            random_board_pairs.append(pickle.load(f))
    except EOFError:
        pass

In [3]:
len(random_board_pairs)

22721

In [4]:
game_board_pairs = []
with open("2021-09-12-mcts-v3-boards11-games.pickle", "rb") as f:
    try:
        while True:
            game_board_pairs.append(pickle.load(f))
    except EOFError:
        pass

In [5]:
len(game_board_pairs)

22702

In [6]:
random_inputs = []
random_outputs = []
for p in random_board_pairs:
    random_inputs.append(p[0].get_flipped_3d_array())
    random_outputs.append(float(p[1]))

In [7]:
game_inputs = []
game_outputs = []
for p in game_board_pairs:
    game_inputs.append(p[0].get_flipped_3d_array())
    game_outputs.append(float(p[1]))

In [8]:
import numpy as np

In [9]:
game_inputs = np.array(game_inputs)
random_inputs = np.array(random_inputs)
game_outputs = np.array(game_outputs)
random_outputs = np.array(random_outputs)

In [10]:
random_mark = int(len(random_outputs)*0.8)

In [11]:
game_mark = int(len(game_outputs)*0.8)

In [12]:
print(random_mark, game_mark)

18176 18161


In [13]:
train_inputs = np.concatenate((random_inputs[0:random_mark] , game_inputs[0:game_mark] 
                               , random_inputs[random_mark:] ,  game_inputs[game_mark:])) 

In [14]:
train_outputs = np.concatenate(( random_outputs[0:random_mark],
                                 game_outputs[0:game_mark],
                                random_outputs[random_mark:] ,
                                game_outputs[game_mark:] ))

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

Maybe I need pooling?? Or an extra Conv2d layer? 

In [16]:
model3 = keras.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [17]:
model3.add(layers.Flatten())
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(1, activation='relu'))

In [19]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 9, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 7488)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1917184   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 1,973,473
Trainable params: 1,973,473
Non-trainable params: 0
______________________________________________

I have fewer parameters? Because the last conv layer made the array narrower than it was tall, so I didn't need as much dense?

In [21]:
model3.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [21]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.1007 - val_loss: 0.0646
Epoch 2/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0521 - val_loss: 0.0523
Epoch 3/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0477 - val_loss: 0.0491
Epoch 4/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0441 - val_loss: 0.0503
Epoch 5/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0418 - val_loss: 0.0490
Epoch 6/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0392 - val_loss: 0.0468
Epoch 7/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0372 - val_loss: 0.0497
Epoch 8/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0350 - val_loss: 0.0485
Epoch 9/10
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0327 - val_loss: 0.0459
Epoch 10/10
1136/1136 [==============================] - 23s 20m

Aiming for below 0.047 CV loss

In [ ]:
model3.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1136/1136 [==============================] - 35s 30ms/step - loss: 0.1009 - val_loss: 0.0562
Epoch 2/10
1136/1136 [==============================] - 34s 30ms/step - loss: 0.0521 - val_loss: 0.0528
Epoch 3/10
 173/1136 [===>..........................] - ETA: 28s - loss: 0.0455

In [22]:
model2.fit(train_inputs, train_outputs, epochs=3, validation_split = 0.2)

Epoch 1/3
1136/1136 [==============================] - 22s 20ms/step - loss: 0.0301 - val_loss: 0.0474
Epoch 2/3
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0286 - val_loss: 0.0472
Epoch 3/3
1136/1136 [==============================] - 23s 20ms/step - loss: 0.0272 - val_loss: 0.0485


Hmm. When training v4 (and the others, I think), the CV loss would be about 0.029. Now it's about 0.047. That's 62% higher. 

Maybe that's a good sign—the data is more complicated and matches the real world better, so the neural net has to struggle to learn it.

Maybe that's a bad sign—the data has a lot of random noise, so the net has to struggle to learn it

Maybe I need a larger model? But let's first see how this one does

In [23]:
model2.save('saved_model/2021-09-15-v5-model')

INFO:tensorflow:Assets written to: saved_model/2021-09-15-v5-model/assets
